# Interactive visualizer
Using [Interactive visualizers](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html) you can bring your own renderer and connect it to the visualizer with live mouse camera control right in the notebook, for example to debug your custom rendering function. The main condition is that the renderer has to take a [Camera](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.camera.camera.html#kaolin-render-camera-camera) as input.

In this notebook, we show how to visualize differentiable rendering of a multi-material mesh from ShapeNet using spherical gaussians lighting. 

In [1]:
import copy
import glob
import math
import logging
import numpy as np
import os
import sys
import torch

from tutorial_common import COMMON_DATA_DIR
import kaolin as kal

import nvdiffrast
glctx = nvdiffrast.torch.RasterizeGLContext(False, device='cuda')

## Load Mesh information

In [2]:
# Set KAOLIN_TEST_SHAPENETV2_PATH env variable, or replace by your shapenet path
SHAPENETV2_PATH = os.getenv('KAOLIN_TEST_SHAPENETV2_PATH')

if SHAPENETV2_PATH is not None:
    ds = kal.io.shapenet.ShapeNetV2(root=SHAPENETV2_PATH,
                                    categories=['car'],
                                    train=True, split=1.,
                                    with_materials=True,
                                    output_dict=True)
    mesh = ds[0]['mesh']
else:
    # Load a specific obj instead
    OBJ_PATH = os.path.join(COMMON_DATA_DIR, 'meshes', 'fox.obj')
    mesh = kal.io.obj.import_mesh(OBJ_PATH, with_materials=True, with_normals=True, triangulate=True)

def process_mesh(mesh):
    # Batch, move to GPU and center and normalize vertices in the range [-0.5, 0.5]
    mesh = mesh.to_batched().cuda()
    mesh.vertices = kal.ops.pointcloud.center_points(mesh.vertices, normalize=True)
    print(mesh)

    diffuse_maps = [m['map_Kd'].unsqueeze(0).cuda().float() / 255. if 'map_Kd' in m else
                    m['Kd'].reshape(1, 1, 1, 3).cuda()
                    for m in mesh.materials[0]]
    specular_maps = [m['map_Ks'].unsqueeze(0).cuda().float() / 255. if 'map_Ks' in m else
                     m['Ks'].reshape(1, 1, 1, 3).cuda()
                     for m in mesh.materials[0]]

    # Use a single diffuse color as backup when map doesn't exist (and face_uvs_idx == -1)
    mesh.uvs = torch.nn.functional.pad(mesh.uvs, (0, 0, 0, 1))
    mesh.face_uvs_idx[mesh.face_uvs_idx == -1] = mesh.uvs.shape[1] - 1
    return mesh, diffuse_maps, specular_maps

mesh, diffuse_maps, specular_maps = process_mesh(mesh)

SurfaceMesh object with batching strategy FIXED
            vertices: [1, 5002, 3] (torch.float32)[cuda:0]  
               faces: [10000, 3] (torch.int64)[cuda:0]  
             normals: [1, 5002, 3] (torch.float32)[cuda:0]  
    face_normals_idx: [1, 10000, 3] (torch.int64)[cuda:0]  
                 uvs: [1, 5505, 2] (torch.float32)[cuda:0]  
        face_uvs_idx: [1, 10000, 3] (torch.int64)[cuda:0]  
material_assignments: [1, 10000] (torch.int16)[cuda:0]  
           materials: [
                      0: list of length 1
                      ]
       face_vertices: if possible, computed on access from: (faces, vertices)
        face_normals: if possible, computed on access from: (normals, face_normals_idx) or (vertices, faces)
            face_uvs: if possible, computed on access from: (uvs, face_uvs_idx)
      vertex_normals: if possible, computed on access from: (faces, face_normals)
     vertex_tangents: if possible, computed on access from: (faces, vertices, face_uvs)


## Instantiate a camera

With the general constructor `Camera.from_args()` the underlying constructors are `CameraExtrinsics.from_lookat()` and `PinholeIntrinsics.from_fov` we will use this camera as a starting point for the visualizers.

In [3]:
camera = kal.render.camera.Camera.from_args(eye=torch.tensor([2., 1., 1.], device='cuda'),
                                            at=torch.tensor([0., 0., 0.]),
                                            up=torch.tensor([1., 1., 1.]),
                                            fov=math.pi * 45 / 180,
                                            width=512, height=512, device='cuda')

## Rendering a mesh

Here we are rendering the loaded mesh with [nvdiffrast](https://github.com/NVlabs/nvdiffrast) using the camera object created above and use both diffuse and specular reflectance for lighting.

For more information on lighting in Kaolin see [diffuse](./diffuse_lighting.ipynb) and [specular](./sg_specular_lighting.ipynb) tutorials and the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.lighting.html).

In [4]:
# Those are the parameters used to define the Spherical gaussian
azimuth = torch.zeros((1,), device='cuda')
elevation = torch.full((1,), math.pi / 3., device='cuda')
amplitude = torch.full((1, 3), 3., device='cuda')
sharpness = torch.full((1,), 5., device='cuda')
# We will use this variable to enable / disable specular reflectance
global apply_specular
apply_specular = True

def generate_pinhole_rays_dir(camera, height, width, device='cuda'):
    """Generate centered grid.
    
    This is a utility function for specular reflectance with spherical gaussian.
    """
    pixel_y, pixel_x = torch.meshgrid(
        torch.arange(height, device=device),
        torch.arange(width, device=device),
        indexing='ij'
    )
    pixel_x = pixel_x + 0.5  # scale and add bias to pixel center
    pixel_y = pixel_y + 0.5  # scale and add bias to pixel center

    # Account for principal point (offsets from the center)
    pixel_x = pixel_x - camera.x0
    pixel_y = pixel_y + camera.y0

    # pixel values are now in range [-1, 1], both tensors are of shape res_y x res_x
    # Convert to NDC
    pixel_x = 2 * (pixel_x / width) - 1.0
    pixel_y = 2 * (pixel_y / height) - 1.0

    ray_dir = torch.stack((pixel_x * camera.tan_half_fov(kal.render.camera.intrinsics.CameraFOV.HORIZONTAL),
                           -pixel_y * camera.tan_half_fov(kal.render.camera.intrinsics.CameraFOV.VERTICAL),
                           -torch.ones_like(pixel_x)), dim=-1)

    ray_dir = ray_dir.reshape(-1, 3)    # Flatten grid rays to 1D array
    ray_orig = torch.zeros_like(ray_dir)

    # Transform from camera to world coordinates
    ray_orig, ray_dir = camera.extrinsics.inv_transform_rays(ray_orig, ray_dir)
    ray_dir /= torch.linalg.norm(ray_dir, dim=-1, keepdim=True)

    return ray_dir[0].reshape(1, height, width, 3)


def base_render(mesh, diffuse_maps, specular_maps, camera, height, width, clear=False):
    """Base function for rendering using separate height and width, assuming batch_size=1"""
    vertices_camera = camera.extrinsics.transform(mesh.vertices)
    face_vertices_camera = kal.ops.mesh.index_vertices_by_faces(
        vertices_camera, mesh.faces)
    face_normals_z = kal.ops.mesh.face_normals(
        face_vertices_camera,
        unit=True
    )[..., -1:].contiguous()

    # Projection: nvdiffrast take clip coordinates as input to apply barycentric perspective correction.
    # Using `camera.intrinsics.transform(vertices_camera) would return the normalized device coordinates.
    proj = camera.projection_matrix()[None]
    homogeneous_vecs = kal.render.camera.up_to_homogeneous(
        vertices_camera
    )[..., None]
    vertices_clip = (proj @ homogeneous_vecs).squeeze(-1)

    rast = nvdiffrast.torch.rasterize(
        glctx, vertices_clip, mesh.faces.int(),
        (height, width), grad_db=False
    )
    # nvdiffrast rasteriztion output is y-up, we need to flip as our display is y-down
    rast0 = torch.flip(rast[0], dims=(1,))
    hard_mask = rast0[:, :, :, -1:] != 0
    face_idx = (rast0[..., -1].long() - 1).contiguous()

    uv_map = nvdiffrast.torch.interpolate(
        mesh.uvs, rast0, mesh.face_uvs_idx[0, ...].int()
    )[0] % 1.
    
    if mesh.has_attribute('normals') and mesh.has_attribute('face_normals_idx'):
        im_world_normals = nvdiffrast.torch.interpolate(
            mesh.normals, rast0, mesh.face_normals_idx[0, ...].int())[0]
    else:
        im_world_normals = nvdiffrast.torch.interpolate(
            mesh.face_normals.reshape(len(mesh), -1, 3), rast0,
            torch.arange(mesh.faces.shape[0] * 3, device='cuda', dtype=torch.int).reshape(-1, 3)
        )[0]
    
    batch_idx = torch.arange(len(mesh), device='cuda', dtype=torch.long).reshape(
        len(mesh), 1, 1).expand(len(mesh), height, width)
    
    im_cam_normals = face_normals_z[batch_idx, face_idx] * (face_idx.unsqueeze(-1) != -1)
    im_world_normals = im_world_normals * torch.sign(im_cam_normals)
    albedo = torch.zeros(
        (1, height, width, 3),
        dtype=torch.float, device='cuda'
    )
    spec_albedo = torch.zeros(
        (1, height, width, 3),
        dtype=torch.float, device='cuda'
    )
    # Obj meshes can be composed of multiple materials
    # so at rendering we need to interpolate from corresponding materials
    im_material_idx = mesh.material_assignments[0, ...][face_idx]
    im_material_idx[face_idx == -1] = -1

    for i, material in enumerate(diffuse_maps):
        mask = im_material_idx == i
        mask_idx = torch.nonzero(mask, as_tuple=False)
        _texcoords = uv_map[mask]
        _texcoords[:, 1] = -_texcoords[:, 1]
        if _texcoords.shape[0] > 0:
            pixel_val = nvdiffrast.torch.texture(
                diffuse_maps[i].contiguous(),
                _texcoords.reshape(1, 1, -1, 2).contiguous(),
                filter_mode='linear'
            )
            albedo[mask] = pixel_val[0, 0]
            pixel_val = nvdiffrast.torch.texture(
                specular_maps[i].contiguous(),
                _texcoords.reshape(1, 1, -1, 2).contiguous(),
                filter_mode='linear'
            )
            spec_albedo[mask] = pixel_val[0, 0] #.permute(1, 0)
    img = torch.zeros((1, height, width, 3),
                      dtype=torch.float, device='cuda')
    sg_x, sg_y, sg_z = kal.ops.coords.spherical2cartesian(azimuth, elevation)
    directions = torch.stack(
        [sg_x, sg_z, sg_y],
        dim=-1
    )
    im_world_normals = im_world_normals[hard_mask.squeeze(-1)]
    diffuse_effect = kal.render.lighting.sg_diffuse_inner_product(
        amplitude, directions, sharpness,
        im_world_normals,
        albedo[hard_mask.squeeze(-1)]
    )
    img[hard_mask.squeeze(-1)] = diffuse_effect
    global apply_specular
    if apply_specular:
        rays_d = generate_pinhole_rays_dir(camera, height, width)
        specular_effect = kal.render.lighting.sg_warp_specular_term(
            amplitude, directions, sharpness,
            im_world_normals,
            torch.full((im_world_normals.shape[0],), 0.5, device='cuda'),
            -rays_d[hard_mask.squeeze(-1)],
            spec_albedo[hard_mask.squeeze(-1)]
        )
        img[hard_mask.squeeze(-1)] += specular_effect

    if clear:
        img = torch.cat([img, hard_mask], dim=-1)  # Add Alpha channel
    final = (torch.clamp(img * hard_mask, 0., 1.)[0] * 255.).to(torch.uint8)
    
    # 'img' is the displayed image, while the other value `face_idx` is only printed on query
    return {
        'img': final,
        'face_idx': face_idx[0]
    }

def render(camera):
    """Render using camera dimension.
    
    This is the main function provided to the interactive visualizer
    """
    return base_render(mesh, diffuse_maps, specular_maps, camera, camera.height, camera.width)

def lowres_render(camera):
    """Render with lower dimension.
    
    This function will be used as a "fast" rendering used when the mouse is moving to avoid slow down.
    """
    return base_render(mesh, diffuse_maps, specular_maps, camera, int(camera.height / 4), int(camera.width / 4))

## Turntable visualizer
This is a simple visualizer useful to inspect a small object.

You can move around with the mouse (left button) and zoom with the mouse wheel.
See the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html#kaolin.visualize.IpyTurntableVisualizer) to customize the sensitivity.

In [5]:
visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render,
    fast_render=lowres_render, max_fps=24, world_up_axis=1)
visualizer.show()

Canvas(height=512, width=512)

Output()

## First person visualizer
This is a visualizer useful to inspect details on an object, or a big scene.

You can move the orientation of the camera with the mouse left button, move the camera around with the mouse right button or
the keys 'i' (up), 'k' (down), 'j' (left), 'l' (right), 'o' (forward), 'u' (backward)

See the [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.visualize.html#kaolin.visualize.IpyFirstPersonVisualizer) to customize the sensitivity and keys.

--------------------
*Note: camera are mutable in the visualizer. If you want to keep track of the camera position you can remove the `copy.deepcopy` on camera argument or you can check `visualizer.camera`*

In [6]:
visualizer = kal.visualize.IpyFirstPersonVisualizer(
    512, 512, copy.deepcopy(camera), render, fast_render=lowres_render,
    max_fps=24, world_up=torch.tensor([0., 1., 0.], device='cuda'))
visualizer.show()

Canvas(height=512, width=512)

Output()

## Adding events and other widgets

The visualizer is modular.
Here we will add:
* sliders to control the spherical gaussian parameters (see [ipywidgets tutorial](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html) for more info).
* A key event to 'space' to enable / disable specular reflectance (see [ipyevents documentation](https://github.com/mwcraig/ipyevents/blob/main/docs/events.ipynb)) to see all the events that can be caught.

In general if you want to modify the rendering function you can use global variables or make a class (with the rendering function being a method)

-------------
More info on spherical gaussians parameters in our [sg_specular_lighting.ipynb](./sg_specular_lighting.ipynb) tutorial
and [documentation](https://kaolin.readthedocs.io/en/latest/modules/kaolin.render.lighting.html).

In [7]:
from ipywidgets import interactive, HBox, FloatSlider

def additional_event_handler(visualizer, event):
    """Event handler to be provided to Kaolin's visualizer"""
    with visualizer.out: # This is for catching print and errors
        if event['type'] == 'keydown' and event['key'] == ' ':
            global apply_specular
            apply_specular = not apply_specular
            visualizer.render_update()
            return False
        return True

visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render,
    fast_render=lowres_render, max_fps=24,
    additional_event_handler=additional_event_handler,
    additional_watched_events=['keydown'] # We need to now watch for key press event
)
# we don't call visualizer.show() here

def sliders_callback(new_elevation, new_azimuth, new_amplitude, new_sharpness):
    """ipywidgets sliders callback"""
    with visualizer.out: # This is in case of bug
        elevation[:] = new_elevation
        azimuth[:] = new_azimuth
        amplitude[:] = new_amplitude
        sharpness[:] = new_sharpness
        # this is how we request a new update
        visualizer.render_update()
        
elevation_slider = FloatSlider(
    value=elevation.item(),
    min=-math.pi / 2.,
    max=math.pi / 2.,
    step=0.1,
    description='Elevation:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

azimuth_slider = FloatSlider(
    value=azimuth.item(),
    min=-math.pi,
    max=math.pi,
    step=0.1,
    description='Azimuth:',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

amplitude_slider = FloatSlider(
    value=amplitude[0,0].item(),
    min=0.1,
    max=20.,
    step=0.1,
    description='Amplitude:\n',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

sharpness_slider = FloatSlider(
    value=sharpness.item(),
    min=0.1,
    max=20.,
    step=0.1,
    description='Sharpness:\n',
    continuous_update=True,
    readout=True,
    readout_format='.1f',
)

interactive_slider = interactive(
    sliders_callback,
    new_elevation=elevation_slider,
    new_azimuth=azimuth_slider,
    new_amplitude=amplitude_slider,
    new_sharpness=sharpness_slider
)

# We combine all the widgets and the visualizer canvas and output in a single display
full_output = HBox([visualizer.canvas, interactive_slider])
display(full_output, visualizer.out)

Output()

## Customizing Drawing and Event Canvases

In some cases, it may be desirable to receive events on a different canvas from the one used for drawing, or you may want to create the drawing canvas manually. 

You can mix and match any number of drawing canvasses and event canvasses by passing `canvas` and `event_canvas` variables to the visualizer constructor. 

In this example, we show:

1. How to use Kaolin visualizers with MultiCanvas, where you may draw something else on one of the stacked sub-canvasses (in this case, background color)
2. How to control visualizer using events from another canvas (in this case, mouse motion in the first canvas controls both cameras)

There are many ways you may want to build debug interfaces around your rendering function, and our visualizers allow for flexibility.

In [8]:
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from ipywidgets import Layout, VBox, Dropdown
from functools import partial

# Create two multi-layer canvasses, containing 2 aligned images each with opacity enabled
# (e.g. useful for various overlays)
# multi_canvas[0] - background canvas
# multi_canvas[1] - foreground canvas
cwidth = 512
multi_canvas = MultiCanvas(2, width=cwidth, height=cwidth, layout=Layout(width="500px", height="500px"))
multi_canvas2 = MultiCanvas(2, width=cwidth, height=cwidth, layout=Layout(width="500px", height="500px"))
colors = ["red", "blue", "green", "purple", "#cc3333"]

# Function to set background canvas of both canvasses
def set_background(val): 
    with hold_canvas():
        multi_canvas[0].fill_style = val
        multi_canvas[0].fill_rect(0, 0, cwidth, cwidth)
        multi_canvas2[0].fill_style = val
        multi_canvas2[0].fill_rect(0, 0, cwidth, cwidth)
        
# Actually set background and clear front canvasses
set_background(colors[0])
with hold_canvas():
    multi_canvas[1].clear_rect(0, 0, cwidth, cwidth)
    multi_canvas2[1].clear_rect(0, 0, cwidth, cwidth)
    
# Create background color picker
def handle_dropdown(change):
    global background_color
    with visualizer.out:
        set_background(change['new'])
color_dropdown = Dropdown(options=colors, value='red', description='Background:')
color_dropdown.observe(handle_dropdown, names='value')


# Read in an additional mesh
mesh2 = kal.io.obj.import_mesh(os.path.join(COMMON_DATA_DIR, 'meshes', 'pizza.obj'), with_materials=True, with_normals=True, triangulate=True)
mesh2, diffuse_maps2, specular_maps2 = process_mesh(mesh2)
    
# Create render closures that only requires a camera to render; ensure we render with Alpha channel
render1 = partial(base_render, mesh, diffuse_maps, specular_maps, width=cwidth, height=cwidth,  clear=True)
lowres_render1 = partial(base_render, mesh, diffuse_maps, specular_maps, width=cwidth//4, height=cwidth//4, clear=True)
render2 = partial(base_render, mesh2, diffuse_maps2, specular_maps2, width=cwidth, height=cwidth,  clear=True)
lowres_render2 = partial(base_render, mesh2, diffuse_maps2, specular_maps2, width=cwidth//4, height=cwidth//4, clear=True)

# Create first visualizer
visualizer = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render1,
    fast_render=lowres_render1, max_fps=24, world_up_axis=1,
    canvas=multi_canvas[1], event_canvas=multi_canvas)

# Create second visualizer, still controlled by mouse within the first canvas
visualizer2 = kal.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(camera), render2,
    fast_render=lowres_render2, max_fps=24, world_up_axis=1,
    canvas=multi_canvas2[1], event_canvas=multi_canvas)

# Show all the canvasses and outputs
visualizer.render_update()
visualizer2.render_update()
VBox((HBox((multi_canvas,multi_canvas2)), color_dropdown, HBox((visualizer.out, visualizer2.out))))

SurfaceMesh object with batching strategy FIXED
            vertices: [1, 482, 3] (torch.float32)[cuda:0]  
               faces: [960, 3] (torch.int64)[cuda:0]  
             normals: [1, 482, 3] (torch.float32)[cuda:0]  
    face_normals_idx: [1, 960, 3] (torch.int64)[cuda:0]  
                 uvs: [1, 514, 2] (torch.float32)[cuda:0]  
        face_uvs_idx: [1, 960, 3] (torch.int64)[cuda:0]  
material_assignments: [1, 960] (torch.int16)[cuda:0]  
           materials: [
                      0: list of length 2
                      ]
       face_vertices: if possible, computed on access from: (faces, vertices)
        face_normals: if possible, computed on access from: (normals, face_normals_idx) or (vertices, faces)
            face_uvs: if possible, computed on access from: (uvs, face_uvs_idx)
      vertex_normals: if possible, computed on access from: (faces, face_normals)
     vertex_tangents: if possible, computed on access from: (faces, vertices, face_uvs)
